In [2]:
from ConvergenceAnalysis import *


In [3]:
fd_beta_data
# rename System to Molecule
fd_beta_data.rename(columns={'System':'Molecule'}, inplace=True)
mols= fd_beta_data.Molecule.unique().tolist()
# remove KF 
mols.remove('KF')
mols.remove('LiBr')


In [4]:
fd_beta_data

In [5]:
# query molecule LiH 
fd_beta_data.query('Molecule == "LiH"')

In [6]:
from DataAnalysisClass import *

october_absolute_kain_path = Path('/mnt/data/madness_data/october_beta_absolute_kain/')
paper_path = Path('/home/adrianhurtado/projects/writing/mra-tdhf-polarizability/Figures_v2')



In [7]:

single = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV5Z', 'aug-cc-pV6Z']
single_polarized = ['aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ']
double = ['d-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z']
double_polarized = ['d-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
all_basis_sets = single + single_polarized + double + double_polarized


In [8]:
low_q_data=QuadraticDatabase(mols,all_basis_sets,'hf','dipole',[0],october_absolute_kain_path
                             .joinpath('low-low'),False)
high_q_data=QuadraticDatabase(mols,all_basis_sets,'hf','dipole',[0],october_absolute_kain_path
                             .joinpath('high-high'),False)
# save data

In [9]:
#low_q_data.save_dfs()
high_q_data.save_dfs()


In [10]:

# get the percent error relative to the FD datal
beta_data=low_q_data.q_df.copy()
beta_data=high_q_data.q_df.copy()

low_z_data=create_z_beta_data(low_q_data.q_df)
# add column called MRA-LOW
low_z_data['Protocol']='Medium'
high_z_data=create_z_beta_data(high_q_data.q_df)
# add column called MRA-HIGH
high_z_data['Protocol']='High'
# concatenate the two dataframes
z_data=pd.concat([low_z_data,high_z_data],axis=0)


z_data


In [11]:

beta_z_plot=compare_z_beta_to_basis_set(z_data,'Absolute Percent Error')
# save the plot
beta_z_plot.savefig(paper_path.joinpath('beta_z_plot_2.png'),dpi=600)


In [12]:
z_data.query('basis=="MRA"').sort_values('molecule')

In [13]:



low_data = PolarizabilityData(mols, 'hf', 'dipole', all_basis_sets,
                              october_absolute_kain_path.joinpath
                              ('low-low'),overwrite=False)
high_data = PolarizabilityData(mols, 'hf', 'dipole', all_basis_sets, october_absolute_kain_path
                               .joinpath('high-high'),overwrite=True)

low_data.save_dfs()
high_data.save_dfs()
low_data.polar_data.query('omega==0')
low_data.iso_data.query('omega==0')


In [14]:
low_z_data=create_z_data(low_data,'two')
# create Protocol column
low_z_data['Protocol']='Medium'
high_z_data=create_z_data(high_data,'two')
high_z_data['Protocol']='High'
z_data=pd.concat([low_z_data,high_z_data])
z_data
# remove HeNe
z_data=z_data.query('molecule!="HeNe"')



In [15]:
z_abs_error_plot=compare_z_to_basis_set(z_data,y='Absolute Percent Error')
z_abs_error_plot.savefig(paper_path.joinpath('z_abs_error_plot_2.png'),dpi=600)


In [16]:
low_data.iso_data.query('omega==0')
# grab the basis set data only 
basis_data=low_data.iso_data.query('omega==0 and basis!="MRA"').copy()
# add a column for the MRA data for each molecule by mapping based on molecule
basis_data['alpha(MRA)']=basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                      'basis=="MRA"').set_index
                                                    ('molecule')['alpha'].to_dict())
basis_data['gamma(MRA)']=basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                      'basis=="MRA"').set_index
                                                    ('molecule')['gamma'].to_dict())
#basis_data
# compute the percent error in alpha and gamma
basis_data['alpha error']=100*(basis_data['alpha']-basis_data['alpha(MRA)'])/basis_data['alpha(MRA)']
basis_data['gamma error']=100*(basis_data['gamma']-basis_data['gamma(MRA)'])/basis_data['gamma(MRA)']


In [17]:
basis_data=make_detailed_df(basis_data)
basis_data


In [18]:
with sns.plotting_context('paper',font_scale=1.5):
    g=sns.catplot(col='valence',x='Type',y='alpha error',hue='mol_system',
                  data=basis_data,kind='strip',dodge=False,jitter=True,col_wrap=2,
                  s=100,palette='colorblind',height=4,aspect=1.5,sharey=False)
    
    for ax in g.axes.flat:
        #ax.set_yscale('symlog', linthresh=1e-2)
        # draw horizontal line at .05%
        ax.axhline(y=.05,linestyle='--',color='orange')
        ax.axhline(y=-.05,linestyle='--',color='orange')
         
        ax.set_ylabel(r'$\alpha$ Error (%)')
        # rotate the xlabels
        for label in ax.get_xticklabels():
            label.set_rotation(45)
            label.set_horizontalalignment('right')
        
        
            
        


In [19]:
low_data.iso_data.query('omega==0')
# grab the basis set data only 
basis_data=low_data.iso_data.query('omega==0 and basis!="MRA"').copy()
# add a column for the MRA data for each molecule by mapping based on molecule
basis_data['alpha(MRA)']=basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                            'basis=="MRA"').set_index
                                                    ('molecule')['alpha'].to_dict())
basis_data['gamma(MRA)']=basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                            'basis=="MRA"').set_index
                                                    ('molecule')['gamma'].to_dict())
#basis_data
# compute the percent error in alpha and gamma
basis_data['alpha error']=100*(basis_data['alpha']-basis_data['alpha(MRA)'])/basis_data['alpha(MRA)']
basis_data['gamma error']=100*(basis_data['gamma']-basis_data['gamma(MRA)'])/basis_data['gamma(MRA)']



In [20]:
basis_data=high_q_data.beta_hrs_df.query('basis!="MRA" & Afreq==0.0').copy()
basis_data['beta(MRA)']=basis_data['molecule'].map(low_q_data.beta_hrs_df.query('basis=="MRA" & '
                                                                                'Afreq==0.0')
                                                   .set_index('molecule')['Beta'].to_dict())
basis_data['beta error']=100*(basis_data['Beta']-basis_data['beta(MRA)'])/basis_data['beta(MRA)']
basis_data=make_detailed_df(basis_data)
basis_data

In [21]:
with sns.plotting_context('paper',font_scale=1.0):
    g=sns.catplot(col='valence',x='Type',y='beta error',hue='mol_system',
                  data=basis_data,kind='strip',dodge=False,jitter=True,col_wrap=2,
                  s=100,palette='colorblind',height=4,aspect=1.5,sharey=False)

    for ax in g.axes.flat:
        #ax.set_yscale('symlog', linthresh=1e-2)
        # draw horizontal line at .05%
        ax.axhline(y=.05,linestyle='--',color='orange')
        ax.axhline(y=-.05,linestyle='--',color='orange')

        ax.set_ylabel(r'$\beta_{HRS}$ Error (%)')
        # rotate the xlabels
        for label in ax.get_xticklabels():
            label.set_rotation(45)
            label.set_horizontalalignment('right')


In [22]:

basis_data=high_q_data.q_df.query('basis!="MRA" & Afreq==0.0 & ijk=="ZZZ"').copy()
basis_data['beta(MRA)']=basis_data['molecule'].map(low_q_data.q_df.query('basis=="MRA" & '
                                                                                'Afreq==0.0 & ijk=="ZZZ"')
                                                   .set_index('molecule')['Beta'].to_dict())
basis_data['beta error']=100*(basis_data['Beta']-basis_data['beta(MRA)'])/basis_data['beta(MRA)']
basis_data=make_detailed_df(basis_data)
basis_data


In [23]:

with sns.plotting_context('paper',font_scale=1.0):
    g=sns.catplot(col='valence',x='Type',y='beta error',hue='mol_system',
                  data=basis_data,kind='strip',dodge=False,jitter=True,col_wrap=2,
                  s=100,palette='colorblind',height=4,aspect=1.5,sharey=False)

    for ax in g.axes.flat:
        #ax.set_yscale('symlog', linthresh=1e-2)
        # draw horizontal line at .05%
        ax.axhline(y=.05,linestyle='--',color='orange')
        ax.axhline(y=-.05,linestyle='--',color='orange')

        ax.set_ylabel(r'$\beta_{ZZZ}$ Error (%)')
        # rotate the xlabels
        for label in ax.get_xticklabels():
            label.set_rotation(45)
            label.set_horizontalalignment('right')

